In [1]:
import os
import yfinance as yf
from datetime import datetime
import pandas as pd
import pytz
import logging

import matplotlib.pyplot as plt

In [2]:
import numpy as np
import plotly.graph_objects as go
from PIL import Image

In [3]:
loaded = np.load('MSFT_test.npz')

In [4]:
dta_x = loaded['a']

In [5]:
dta_x.shape

(500, 700, 8568)

In [6]:
dta_y = loaded['b']
dta_y.shape

(8568,)

## Pytorch Machine Learning Shit

In [7]:
from ResNet_CNN import *

In [8]:
import math as m
import torch
from torch.nn import Linear, ReLU, Conv1d, Conv2d, Flatten, Sequential, CrossEntropyLoss, MSELoss, MaxPool1d, MaxPool2d, Dropout, BatchNorm1d, BatchNorm2d

from torch.optim import Adam
from torch import nn
import torchvision

In [9]:
from functools import partial
from collections import OrderedDict

In [10]:
import torchvision.models as models

In [11]:
from torchsummary import summary

In [12]:
model = res_conv1(1, 500)

In [13]:
print(model)

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNetBottleNeckBlock(
            (blocks): Sequential(
              (0): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): ReLU(inplace=True)
              (2): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

## Putting everything together

In [15]:
X = dta_x[:,:,568:768]

In [16]:
Y = dta_y[568:768]

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
import torchvision.transforms as transforms

In [19]:
trans = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [21]:
from torch.autograd import Variable
from torch.optim import Adam, SGD

In [22]:
X = X.reshape((-1, 500, 700)).astype(np.float32)
Y = Y.reshape((-1, 1)).astype(np.float32)

In [23]:
X.shape

(200, 500, 700)

In [24]:
Y.shape

(200, 1)

In [25]:
train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size=0.1, random_state=42)

In [26]:
(train_X.shape, train_Y.shape), (val_X.shape, val_Y.shape)

(((180, 500, 700), (180, 1)), ((20, 500, 700), (20, 1)))

In [27]:
train_X = train_X.reshape(-1, 1, 500, 700)
train_X = torch.from_numpy(train_X)

train_Y = train_Y.astype(int)
train_Y = torch.from_numpy(train_Y)

In [28]:
val_X =  val_X.reshape(-1, 1, 500, 700)
val_X = torch.from_numpy(val_X)

val_Y = val_Y.astype(int)
val_Y = torch.from_numpy(val_Y)

In [29]:
model = res_conv1(1, 500)

In [30]:
lr = 0.01
optimizer = Adam(model.parameters(), lr=lr)

criterion = CrossEntropyLoss()

Don't run because CUDA is out of memory

In [72]:
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNetBottleNeckBlock(
            (blocks): Sequential(
              (0): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): ReLU(inplace=True)
              (2): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [31]:
def train(epochs):
    model.train()
    tr_loss = 0
    # dataset
    x_train, y_train = Variable(train_X), Variable(train_Y)
    x_val, y_val = Variable(val_X), Variable(val_Y)
 
    optimizer.zero_grad()
    
    output_train = model(x_train)
    output_val = model(x_val)
    
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    
    train_losses.append(loss_train)
    val_losses.append(loss_val)
    
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    
    if epochs % 2 == 0:
        print('Epoch: ', epochs+1, '\t', 'train loss: ', loss_train, '\t', 'val loss: ', loss_val)

In [32]:
n_epochs = 10
train_losses = []
val_losses = []

for epochs in range(n_epochs):
    train(epochs)

RuntimeError: 1D target tensor expected, multi-target not supported